# Example Usage of Scraipe Library

Here's a quick example using scraipe to extract mentions of celebrities in news articles.

## Setup
Install and import things we need. We will use `NewsScraper` and `OpenAiAnalyzer` from the `scraipe[extras]` subpackage.

We will also load [your OpenAI API key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key) from a file called 'openai_key.txt'. This file should be in the same folder as this notebook.

In [1]:
# Install Scraipe
%pip install scraipe[extras] --quiet --upgrade
# Check package version
!pip show scraipe | grep Version

Note: you may need to restart the kernel to use updated packages.
Version: 0.1.25


In [2]:

# Import modules
import pandas as pd
from scraipe import Workflow
from scraipe.extras import NewsScraper, OpenAiAnalyzer
from pydantic import BaseModel

# Load OpenAI API key
OPENAI_API_KEY = open("openai_key.txt").read().strip()

## Extract links
First, we need a list of links to target with scraipe. We will extract all links from the front page of https://apnews.com.

In [3]:
import requests
import re

url = "https://apnews.com/"
response = requests.get(url)
html = response.text

# Use a regex to extract article links
pattern = r'href=["\'](?=[^"\']*/article)([^"\']+)["\']'
news_links = re.findall(pattern, html)

# Remove duplicates
news_links = list(set(news_links))

# Display a summary of the links
news_links_df = pd.DataFrame(news_links, columns=['link'])
import time
print(f"Found {len(news_links_df)} front page AP News links on {time.strftime('%Y-%m-%d')}")
display(news_links_df.head())

Found 135 front page AP News links on 2025-04-04


,link
0,https://apnews.com/article/cooking-with-kids-w...
1,https://apnews.com/article/whale-watching-dog-...
2,https://apnews.com/article/tariffs-timeline-tr...
3,https://apnews.com/article/israel-palestinians...
4,https://apnews.com/article/measles-outbreak-te...


## Configure Workflow
Now we'll configure the scraipe workflow using `NewsScraper` and `OpenAiAnalyzer`.

`NewsScraper` uses `trafilatura` to extract article content from a news site without all the HTML clutter.

`OpenAiAnalyzer` uses OpenAI models to extract data from the article content.

In [4]:
#===Configure NewsScraper===
# NewsScraper doesn't require any additional configuration
scraper = NewsScraper()

#===Configure OpenAiAnalyzer===
# Define the instruction for the LLM. Ensure the instruction specifies a return schema.
instruction = '''
Extract a list of celebrities mentioned in the article text.
Return a JSON dictionary with the following schema:
{"celebrities":["celebrity1", "celebrity2", ...]}
'''

# (Optional) Create a pydantic schema to validate the LLM output
from typing import List
class ExpectedOutput(BaseModel):
    celebrities: List[str]
    
# Create the analyzer with the API key, instruction, and schema
analyzer = OpenAiAnalyzer(OPENAI_API_KEY,instruction,pydantic_schema=ExpectedOutput)

#===Create Workflow===
# Create a workflow with the configured scraper and analyzer
workflow = Workflow(scraper, analyzer)

## Scrape links
Next we will scrape content from news links. This content will be saved within the workflow's scrape store.

In [5]:
# Scrape the news links
workflow.scrape(news_links)
# Display the scraped content
scrape_store_df = workflow.get_scrapes()

display(scrape_store_df.head())

Scraping: 100%|██████████| 135/135 [00:41<00:00,  3.22link/s]


,link,content,scrape_success,scrape_error
0,https://apnews.com/article/proxy-voting-discha...,Speaker Johnson suffers a defeat in his push t...,True,None
1,https://apnews.com/article/donald-trump-speake...,Trump says he supports proxy voting for new pa...,True,None
2,https://apnews.com/article/south-korea-yoon-ma...,Yoon Suk Yeol had a rapid rise in South Korean...,True,None
3,https://apnews.com/article/greenland-denmark-p...,Danish leader tells the US ‘you cannot annex a...,True,None
4,https://apnews.com/article/keukenhof-tulip-flo...,Flower lovers and influencers flock to the tul...,True,None


## Analyze content
Next we will extract celebrities mentioned in each article using OpenAI

In [6]:
# Analyze the scraped content
workflow.analyze()

# Display the analyses
analysis_store_df = workflow.get_analyses()
display(analysis_store_df.head())

Analyzing: 100%|██████████| 135/135 [00:54<00:00,  2.48item/s]


,link,output,analysis_success,analysis_error
0,https://apnews.com/article/proxy-voting-discha...,"{'celebrities': ['Mike Johnson', 'Anna Paulina...",True,None
1,https://apnews.com/article/donald-trump-speake...,"{'celebrities': ['Donald Trump', 'Mike Johnson...",True,None
2,https://apnews.com/article/south-korea-yoon-ma...,"{'celebrities': ['Yoon Suk Yeol', 'Park Geun-h...",True,None
3,https://apnews.com/article/greenland-denmark-p...,"{'celebrities': ['Marco Rubio', 'Donald Trump'...",True,None
4,https://apnews.com/article/keukenhof-tulip-flo...,{'celebrities': []},True,None


## Compile the results
Finally, let's export the completed analysis. 

In [9]:
export_df = workflow.export()
display(export_df)
export_df.to_csv('celebrities.csv', index=False)

,link,scrape_success,analysis_success,celebrities
0,https://apnews.com/article/proxy-voting-discha...,True,True,"[Mike Johnson, Anna Paulina Luna, Brittany Pet..."
1,https://apnews.com/article/donald-trump-speake...,True,True,"[Donald Trump, Mike Johnson, Anna Paulina Luna..."
2,https://apnews.com/article/south-korea-yoon-ma...,True,True,"[Yoon Suk Yeol, Park Geun-hye, Moon Jae-in, Le..."
3,https://apnews.com/article/greenland-denmark-p...,True,True,"[Marco Rubio, Donald Trump, JD Vance, Mette Fr..."
4,https://apnews.com/article/keukenhof-tulip-flo...,True,True,[]
...,...,...,...,...
130,https://apnews.com/article/trump-tariffs-econo...,True,True,"[Donald Trump, Joe Biden, Olu Sonola, Julian E..."
131,https://apnews.com/article/jobs-economy-unempl...,True,True,"[Donald Trump, Elon Musk, Diane Swonk, Seema S..."
132,https://apnews.com/article/severe-weather-thin...,True,True,[]
133,https://apnews.com/article/latinoamerica-carib...,True,True,"[Olivia Rodrigo, Emily Rosenfield]"


## Analyze the results
Now you can conduct your own analysis on the structured data collected by the scraipe workflow.

In [10]:
# Load the extracted data
celebrities_df = pd.read_csv('celebrities.csv')
from ast import literal_eval
celebrities_df['celebrities'] = celebrities_df['celebrities'].apply(literal_eval)

# Explode the nested list of celebrities
celebrities_df = celebrities_df.explode('celebrities')
celebrities_df['celebrities'] = celebrities_df['celebrities'].str.strip()

# Display the top 10 most mentioned celebrities
celebrities_df = celebrities_df['celebrities'].value_counts().reset_index()
celebrities_df.columns = ['celebrity', 'mentions']
celebrities_df = celebrities_df.sort_values('mentions', ascending=False)
celebrities_df.head(10)

,celebrity,mentions
0,Donald Trump,43
1,Joe Biden,11
2,Elon Musk,6
4,Marco Rubio,5
3,Vladimir Putin,5
5,Pete Hegseth,3
6,Yoon Suk Yeol,3
7,Laura Loomer,3
8,Ursula von der Leyen,3
9,Robert F. Kennedy Jr.,3
